In [1]:
import cv2
import os
import numpy as np
import scipy.io as sio
# from google.colab.patches import cv2_imshow

In [2]:
# from google.colab import drive
# drive.mount('/tmp/drive')

# # base directory
base_path = "./"
# # change the current working directory to base_path

In [3]:
print("[INFO] loading edge detector...")
protoPath = os.path.join(base_path, "hed_checkpoints/deploy.prototxt")
modelPath = os.path.join(base_path, "hed_checkpoints/hed_pretrained_bsds.caffemodel")
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

[INFO] loading edge detector...


In [4]:
target_name = "maple_character4"

In [5]:
# %cd /tmp/drive/MyDrive/colab/images

In [6]:
# !unzip -qq "/tmp/drive/MyDrive/colab/images/0.zip"

In [7]:
cv2.cuda.getCudaEnabledDeviceCount()

0

In [8]:
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [9]:
class CropLayer(object):
    def __init__(self, params, blobs):
        # initialize our starting and ending (x, y)-coordinates of the crop
        self.startX = 0
        self.startY = 0
        self.endX = 0
        self.endY = 0

    def getMemoryShapes(self, inputs):
        # the crop layer will receive two inputs -- we need to crop
        # the first input blob to match the shape of the second one,
        # keeping the batch size and number of channels
        (inputShape, targetShape) = (inputs[0], inputs[1])
        (batchSize, numChannels) = (inputShape[0], inputShape[1])
        (H, W) = (targetShape[2], targetShape[3])

        # compute the starting and ending crop coordinates
        self.startX = int((inputShape[3] - targetShape[3]) / 2)
        self.startY = int((inputShape[2] - targetShape[2]) / 2)
        self.endX = self.startX + W
        self.endY = self.startY + H

        # return the shape of the volume (we'll perform the actual
        # crop during the forward pass)
        return [[batchSize, numChannels, H, W]]

    def forward(self, inputs):
        # use the derviced (x, y)-coordinates to perform the crop
        return [inputs[0][:, :, self.startY:self.endY,
                                self.startX:self.endX]]

In [10]:
cv2.dnn_registerLayer("Crop", CropLayer)

In [11]:
len(os.listdir(os.path.join(base_path, f"images/{target_name}/")))

13416

In [12]:
image_directory_path = os.path.join(base_path, f"images/{target_name}/")
# image_directory_path = "/content/bmw"

pad_size = 32

for (index, image_path) in enumerate(os.listdir(image_directory_path)):

    image = cv2.imread(os.path.join(image_directory_path, image_path))
    image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), 'reflect')
    # image = cv2.resize(image, (480, 480))
    # print(f"{image} shape : {image.shape}")
    (H, W) = image.shape[:2]

    # normalize it

    blob = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(W, H),
                            mean=(104.00698794, 116.66876762, 122.67891434),
                            swapRB=False,
                            crop=False)

    net.setInput(blob)
    hed = net.forward()
    hed = cv2.resize(hed[0, 0], (W, H))
    # hed = (255 * hed).astype("uint8")

    # print(hed.shape)
    # print(hed)
    # print(f'max : {np.max(hed)}')
    # print(f'min : {np.min(hed)}')

    # hed = np.where(hed>25, 255, 0)
    # hed = hed.astype(np.uint8)

    image = image[pad_size:-pad_size, pad_size:-pad_size]
    hed = hed[pad_size:-pad_size, pad_size:-pad_size]

    # sio.savemat(os.path.join(base_path, 'mat_images/' + image_path[:-4] + '_edge.mat'), {'edge_predict': hed})
    sio.savemat(os.path.join(base_path, f'mat_images/{target_name}/' + image_path.split('.')[0] + '_edge.mat'), {'edge_predict': hed})

    if index % 100 == 0:
        print(f"processing {index+1}th")
        # cv2_imshow(image)
        # cv2_imshow((255 * hed).astype("uint8"))

processing 1th
processing 101th
processing 201th
processing 301th
processing 401th
processing 501th
processing 601th
processing 701th
processing 801th
processing 901th
processing 1001th
processing 1101th
processing 1201th
processing 1301th
processing 1401th
processing 1501th
processing 1601th
processing 1701th
processing 1801th
processing 1901th
processing 2001th
processing 2101th
processing 2201th
processing 2301th
processing 2401th
processing 2501th
processing 2601th
processing 2701th
processing 2801th
processing 2901th
processing 3001th
processing 3101th
processing 3201th
processing 3301th
processing 3401th
processing 3501th
processing 3601th
processing 3701th
processing 3801th
processing 3901th
processing 4001th
processing 4101th
processing 4201th
processing 4301th
processing 4401th
processing 4501th
processing 4601th
processing 4701th
processing 4801th
processing 4901th
processing 5001th
processing 5101th
processing 5201th
processing 5301th
processing 5401th
processing 5501th
proc